# 1. Recommendation system

In [1]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest
import csv
from ydata_profiling import ProfileReport
from itertools import chain


In [2]:
df = pd.read_csv('vodclickstream_uk_movies_03.csv')

df.head()


,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


### A little bit of data cleaning first

In [3]:
df["datetime"]=pd.to_datetime(df["datetime"])
df["release_date"]=pd.to_datetime(df["release_date"],errors="coerce")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671736 entries, 0 to 671735
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    671736 non-null  int64         
 1   datetime      671736 non-null  datetime64[ns]
 2   duration      671736 non-null  float64       
 3   title         671736 non-null  object        
 4   genres        671736 non-null  object        
 5   release_date  641432 non-null  datetime64[ns]
 6   movie_id      671736 non-null  object        
 7   user_id       671736 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 41.0+ MB


We create a new column containing the genres for each row in a list instead of a string like in the "genres" column

In [4]:
df['genres_list'] = df.genres.apply(lambda row: [word.strip() for word in row.split(',')])
df

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id,genres_list
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe,"[Comedy, Drama, Romance]"
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510,"[Fantasy, Horror, Mystery, Thriller]"
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf,"[Action, Thriller]"
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6,"[Action, Drama]"
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287,"[Animation, Action, Adventure, Comedy, Family,..."
...,...,...,...,...,...,...,...,...,...
671731,730504,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,Talk-Show,2019-06-12,43cd23f30f,57501964fd,[Talk-Show]
671732,730505,2019-06-30 21:49:34,91157.0,HALO Legends,"Animation, Action, Adventure, Family, Sci-Fi",2010-02-16,febf42d55f,d4fcb079ba,"[Animation, Action, Adventure, Family, Sci-Fi]"
671733,730506,2019-06-30 22:00:44,0.0,Pacific Rim,"Action, Adventure, Sci-Fi",2013-07-12,7b15e5ada1,4a14a2cd5a,"[Action, Adventure, Sci-Fi]"
671734,730507,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"Documentary, Music",2019-02-08,52d49c515a,0b8163ea4b,"[Documentary, Music]"


Here we created a set containing all the unique genres in the dataframe

In [5]:
genres=df["genres"].to_frame()
uni=[]
for i in range(len(genres)):
    x=genres["genres"][i].split(', ')
    for j in x:
        uni.append(j)

uni=set(uni)
uni


{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'NOT AVAILABLE',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

We remove the genre : NOT AVAILABLE

In [6]:
uni.remove("NOT AVAILABLE")
uni

{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [7]:
df = df.drop('Unnamed: 0', axis=1)


In [8]:
x="NOT AVAILABLE"
for i in df["genres_list"]:
    if x in i:
        i.remove(x)

# WE removed "NOT AVAILABLE" from the data frame


This is the data frame we will use for the Recommendation system

In [9]:
df

,datetime,duration,title,genres,release_date,movie_id,user_id,genres_list
0,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe,"[Comedy, Drama, Romance]"
1,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510,"[Fantasy, Horror, Mystery, Thriller]"
2,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf,"[Action, Thriller]"
3,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6,"[Action, Drama]"
4,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287,"[Animation, Action, Adventure, Comedy, Family,..."
...,...,...,...,...,...,...,...,...
671731,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,Talk-Show,2019-06-12,43cd23f30f,57501964fd,[Talk-Show]
671732,2019-06-30 21:49:34,91157.0,HALO Legends,"Animation, Action, Adventure, Family, Sci-Fi",2010-02-16,febf42d55f,d4fcb079ba,"[Animation, Action, Adventure, Family, Sci-Fi]"
671733,2019-06-30 22:00:44,0.0,Pacific Rim,"Action, Adventure, Sci-Fi",2013-07-12,7b15e5ada1,4a14a2cd5a,"[Action, Adventure, Sci-Fi]"
671734,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"Documentary, Music",2019-02-08,52d49c515a,0b8163ea4b,"[Documentary, Music]"


In [44]:
ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

We created a dictionary containing all the single Netflix users linked to the genres they watch on the streming platform

In [10]:
user_genres={}

for i in range(len(df)):
    id=df["user_id"][i]
    gen=set(df["genres_list"][i])
    if id not in user_genres:
        user_genres[id]=gen
    user_genres[id].update(gen)


In [11]:
len(user_genres)

# 161918 different users

161918

### EXERCISE 1.1

In [13]:
# We count the number of clicks = "count" for each film and user
sorted_df=(df.groupby(["user_id","movie_id"]).size().reset_index(name="count")).sort_values(by=["user_id","count"],ascending=[True,False])
# We keep just the top 10 most clicked films
top_10 =sorted_df.groupby("user_id").head(10)
# Now we proceed by merging the top_10 datframe with the "title" and "genre" info about the movies
merged_df = pd.merge(top_10, df[["user_id","movie_id","title", "genres"]], on=["movie_id","user_id"], how="left")
merged_df # There are a lot of duplicates

,user_id,movie_id,count,title,genres
0,00004e2862,9bfee795ff,1,Hannibal,"Crime, Drama, Thriller"
1,000052a0a0,4718f9963c,9,Looper,"Action, Drama, Sci-Fi, Thriller"
2,000052a0a0,4718f9963c,9,Looper,"Action, Drama, Sci-Fi, Thriller"
3,000052a0a0,4718f9963c,9,Looper,"Action, Drama, Sci-Fi, Thriller"
4,000052a0a0,4718f9963c,9,Looper,"Action, Drama, Sci-Fi, Thriller"
...,...,...,...,...,...
613389,fffeac83be,a2d8a56924,1,Son of Saul,"Drama, War"
613390,fffeac83be,dda0eae17b,1,Enemy at the Gates,"Drama, History, War"
613391,ffff2c5f9e,6467fee6b6,1,Hot Fuzz,"Action, Comedy, Mystery, Thriller"
613392,ffff2c5f9e,9ab62a3f2c,1,Forks Over Knives,Documentary


In [63]:
merged_df=merged_df.drop_duplicates()
merged_df

,user_id,movie_id,count,title,genres
0,00004e2862,9bfee795ff,1,Hannibal,"Crime, Drama, Thriller"
1,000052a0a0,4718f9963c,9,Looper,"Action, Drama, Sci-Fi, Thriller"
10,000052a0a0,4fa0b092d6,3,Jumanji,"Adventure, Comedy, Family, Fantasy"
13,000052a0a0,7314699c23,3,Frailty,"Crime, Drama, Thriller"
16,000052a0a0,6275614f9a,2,Resident Evil,"Action, Horror, Sci-Fi"
...,...,...,...,...,...
613389,fffeac83be,a2d8a56924,1,Son of Saul,"Drama, War"
613390,fffeac83be,dda0eae17b,1,Enemy at the Gates,"Drama, History, War"
613391,ffff2c5f9e,6467fee6b6,1,Hot Fuzz,"Action, Comedy, Mystery, Thriller"
613392,ffff2c5f9e,9ab62a3f2c,1,Forks Over Knives,Documentary


The df above shows for each user the top 10 films he saw with the relative number of clicks, the movies' genres and title.

### EXERCISE 1.2

Now our aim is to create from scratch a minhash function in order to make users with similar interests in a genre appear in the same bucket.

In [15]:
signature_matrix=pd.DataFrame(index=user_genres.keys(),columns=sorted(list(uni)))
signature_matrix

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
1dea19f6fe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
544dcbc510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7cbcc791bf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ebf43c36b6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a57c992287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45414be0ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
783ec67e84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89c715f3a4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9207e1499b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
for user, genres in user_genres.items():
    for i in genres:
        signature_matrix.loc[user][i]=1

The matrix we are creating has for each user a 0 for a genre he didn't watch and a 1 for each genre he watched

In [17]:
signature_matrix=signature_matrix.fillna(0)
signature_matrix

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
1dea19f6fe,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
544dcbc510,0,1,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
7cbcc791bf,1,1,1,0,1,1,0,0,1,1,...,0,0,1,0,0,0,0,1,0,0
ebf43c36b6,1,1,1,1,1,1,0,1,1,1,...,0,0,0,1,0,0,0,1,0,0
a57c992287,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45414be0ec,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
783ec67e84,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
89c715f3a4,0,1,1,0,1,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
9207e1499b,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


The next step for our minhash function is to make a number n_perm of permutations on the columns. We select n_perm=15

In [18]:
sign=[]
num_perm=15
for i in range(num_perm):
    colonne_permutate = np.random.permutation(signature_matrix.columns)
    mat=signature_matrix[colonne_permutate]
    lista=[]
    for i in user_genres.keys():
        lista.append(np.argmax(mat.loc[i])+1)
    sign.append(lista)

The matrix we obtained is a signature matrix where every column represent a permutation and the numbers represent the index where the first 1 appears

In [19]:
df=pd.DataFrame(sign).T
df = df.set_index(pd.Index(user_genres.keys()))
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1dea19f6fe,3,5,10,21,5,10,14,16,7,6,8,11,6,3,14
544dcbc510,3,5,2,6,5,2,1,3,7,1,3,3,3,2,3
7cbcc791bf,4,5,5,3,6,1,2,2,2,1,4,3,6,1,1
ebf43c36b6,3,3,5,3,5,1,2,2,2,1,4,3,6,1,1
a57c992287,2,2,1,2,1,1,1,2,2,1,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45414be0ec,18,5,10,21,22,10,14,16,8,6,11,11,6,3,14
783ec67e84,3,3,9,5,4,12,8,6,7,4,7,1,10,9,5
89c715f3a4,8,5,5,3,6,1,2,2,2,1,4,3,6,1,1
9207e1499b,3,3,9,20,5,6,12,4,3,4,8,16,10,9,11


In [20]:
# We will divide the matrix above in 4 different sections
b1 =df.iloc[:, :3]
b2 = df.iloc[:, 3:6]
b3 = df.iloc[:, 6:9]
b4 = df.iloc[:, 9:12]
b5 = df.iloc[:, 12:]
bands=[b1,b2,b3,b4,b5]

 We created an empty dictionary called buck and then we updated it by putting as keys the tuples made by the numbers of the first "one" in the 
 singature matrix for each permutation in the bands b1,b2,b3,b4 and b5, and to each of these keys we linked a list of all the users with that tuple.
 By doing that we can now compare users and the genres they like much faster.

In [21]:
buck={}
for b in bands:
    for i in range(len(df)):
        sim=tuple(b.iloc[i])
        if sim not in buck.keys():
            buck[sim]=list()
            buck[sim].append(b.index[i])
        else:
            buck[sim].append(b.index[i])



In [22]:
print("We created "+ str(len(buck)) +" different buckets")

# Due to the casuality of the permutations the number of buckets created changes each time we run the code

We created 1554 different buckets


**EXERCISE 1.3**

In [35]:
user_genres
examples=["ebf43c36b6","7cbcc791bf","57501964fd","45414be0ec","89c715f3a4","b15926c011"]

# We defined the variable examples that we will use for testing

In [24]:
def possible_matches(user_id):
    possible_matchs=[]
    for i in buck.keys():
        if user_id in buck[i]:
            possible_matchs.append(buck[i])

    return set(chain(*possible_matchs))


# This function finds for the user_id queried, all the users he is most likely similar to, by looking at the buckets where the user is located

In [25]:
ex1=possible_matches(examples[0])
ex2=possible_matches(examples[1])
ex3=possible_matches(examples[2])
ex4=possible_matches(examples[3])
len(ex1),len(ex2),len(ex3),len(ex4)

# We tested the function on 4 randomly picked user_ids

(18833, 17841, 20, 12466)

To measure the similarity between users we decided to use the jaccard similarity as suggested also on the book. The function above computes the jaccard similarity 
between two sets

In [26]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union


 Once that we found the set with all the possible users similar to the one queried, we can work just on this set instead of using all the ids, 
 in order to reduce computational time

In [93]:

def top2(user_id):
    rec=pd.DataFrame()
    sol=[]
    lik=user_genres[user_id]
    pm=possible_matches(user_id)
    pm.remove(user_id)
    for i in pm:
        lik2=user_genres[i]
        sim=jaccard_similarity(lik,lik2)
        sol.append(sim)
    rec.index=pm
    rec["js"]=sol
    return rec.sort_values("js",ascending=False)
    

The top2() function returns all the users similar to one queired using the jaccard similarity, a jaccard similarity=1 means that the users like the same genres according to the mihash function reduction

In [94]:
top2(examples[0])


,js
fe7c69b00f,1.000000
d5c7115254,1.000000
b05fb952ba,1.000000
69d2636b5e,1.000000
210fe394e3,1.000000
...,...
56ef9e10af,0.066667
93fc2d5a33,0.066667
78051dcd3c,0.062500
641f18e9d4,0.062500


### EXERCISE 1.3

We wanto to recommend at most five movies to the user to watch based on the movies clicked by similar users. We will follow this guideline: 
* Identify the two most similar users to this user. (using top2())
* If these two users have any movies in common, recommend those movies based on the total number of clicks by these users.
* If there are no more common movies, try to propose the most clicked movies by the most similar user first, followed by the other user.


In [84]:
top2(examples[1])[0:5] # we look at the top 5 results

# if there are some exequo we choose casually one of the user (for example the first one computed)

,js
b3b1dddf1e,1.0
acb4180ba2,1.0
7eb46aeec3,1.0
bbc4102c90,1.0
d19c51332f,1.0


We look at the movies seen by the top2 users

In [85]:
user_1=merged_df[merged_df["user_id"]=="b3b1dddf1e"]
user_1

,user_id,movie_id,count,title,genres
430178,b3b1dddf1e,252c41b823,2,Kung Fu Panda,"Animation, Action, Adventure, Comedy, Family"
430180,b3b1dddf1e,5723d3932d,2,Shrek the Third,"Animation, Adventure, Comedy, Family, Fantasy"
430182,b3b1dddf1e,1f95de977d,1,Grimsby,"Action, Adventure, Comedy, Crime, Thriller"
430183,b3b1dddf1e,30ab175630,1,When We First Met,"Comedy, Fantasy, Romance"
430184,b3b1dddf1e,b845cd10ea,1,Over the Hedge,"Animation, Adventure, Comedy, Family, Fantasy"
430185,b3b1dddf1e,be0d6bcabe,1,Kung Fu Panda 2,"Animation, Action, Adventure, Comedy, Family, ..."
430186,b3b1dddf1e,d8fe485619,1,Shrek Forever After,"Animation, Adventure, Comedy, Family, Fantasy,..."


In [86]:
user_2=merged_df[merged_df["user_id"]=="acb4180ba2"]
user_2

,user_id,movie_id,count,title,genres
413618,acb4180ba2,819a646a75,2,Accepted,Comedy
413620,acb4180ba2,039d9da1ad,1,The 40-Year-Old Virgin,"Comedy, Romance"
413621,acb4180ba2,1f95de977d,1,Grimsby,"Action, Adventure, Comedy, Crime, Thriller"
413622,acb4180ba2,b845cd10ea,1,Over the Hedge,"Animation, Adventure, Comedy, Family, Fantasy"


In [81]:
rec_movies=[]
film_1=list(user_1["title"].values)
film_2=list(user_2["title"].values)
for i in film_1:
    if i in film_2:
        rec_movies.append(i)

rec_movies


['Grimsby', 'Over the Hedge']

Now after these tests we create the function to return the 5 movies recommended

In [118]:
def recommend_film(user_id):
    rec_movies=[]
    d=top2(user_id)
    user_1=merged_df[merged_df["user_id"]==d.index[0:2].values[0]] 
    user_2=merged_df[merged_df["user_id"]==d.index[0:2].values[1]]
    film_1=list(user_1["title"].values)
    film_2=list(user_2["title"].values)
    for i in film_1:
        if i in film_2:
            rec_movies.append(i)
    for j in film_1:
        if len(rec_movies)==5:
            break
        if j not in rec_movies:
            rec_movies.append(j)
    for k in film_2:
        if len(rec_movies)==5:
            break
        if k not in rec_movies:
            rec_movies.append(k)

         
    return rec_movies

# The functions looks at first to the common movies and append them to the rec_movies list then if the length of rec_movies is lower 
# than 5 it looks to the most clicked films by user_1 and then user_2

We run a few examples looking for any possible error

In [119]:
recommend_film(examples[1])

['Grimsby',
 'Over the Hedge',
 'Kung Fu Panda',
 'Shrek the Third',
 'When We First Met']

In [120]:
recommend_film(examples[0])

['Hotel Transylvania 2',
 'The Amazing Spider-Man 2',
 'Bernie',
 'Mercury Rising',
 'Step Brothers']

In [121]:
recommend_film(examples[2])

['Oprah Presents When They See Us Now']

In [122]:
recommend_film(examples[5])

['Love, Rosie',
 'Coin Heist',
 'The Edge of Seventeen',
 'Minor Details',
 'Before I Fall']

**************

# 2. Grouping Users together!

*****

# 5. Algorithmic Question (AQ)

a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on because he wants to do paid counseling for other desperate students: in a recursive fashion, the helped helps the helpable.

In [145]:
def High_score(current_score, exam_scores, index=1):
    if len(exam_scores) == 1:
        return exam_scores[0]

    if index == 1 and len(exam_scores) % 2 == 0:
        index += 1

    if index % 2 != 0:
        next_score = max(exam_scores)
        adjustment = next_score - current_score
        sorted_exam_scores = sorted(exam_scores, reverse=True)
        
        remaining_scores = []
        for num in sorted_exam_scores[1:]:
            remaining_scores.append(num - adjustment)
    else:
        next_score = min(exam_scores)
        adjustment = current_score - next_score
        sorted_exam_scores = sorted(exam_scores)
        
        remaining_scores = []
        for num in sorted_exam_scores[1:]:
            remaining_scores.append(num + adjustment)

    return High_score(next_score, remaining_scores, index + 1)

# Example inputs
initial_score1 = 8
exam_scores1 = [5, 7, 1]
result1 = High_score(initial_score1, exam_scores1)
print("Output 1 result =", result1)

initial_score2 = 25
exam_scores2 = [18, 24, 21, 32, 27]
result2 = High_score(initial_score2, exam_scores2)
print("Output 2 result =", result2)

initial_score3 = 30
exam_scores3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result3 = High_score(initial_score3, exam_scores3)
print("Output 3 result =", result3)

Output 1 result = 11
Output 2 result = 44
Output 3 result = 205


b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university!



The time complexity of the code depends principally from: 
* the sorted() function has a complexity of O(n log(n))
* the iterations have a linear complexity of O(n)



So the final complexity of the High_score function is O(n log(n)) which is a good result

c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)

* The result obrìtained before is good as it has a good O notation, i tried to implement the  "memoization" technique but i didn't manage to improve the time complexity. The complexity of the previous function could be considered fast enough to keep it as a final answer

d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!



In [154]:
def High_score_memo(current_score, exam_scores, index=1, memo={}):
    if len(exam_scores) == 1:
        return exam_scores[0]

    if index == 1 and len(exam_scores) % 2 == 0:
        index += 1

    if (current_score, index, tuple(exam_scores)) in memo:
        return memo[(current_score, index, tuple(exam_scores))]

    if index % 2 != 0:
        next_score = max(exam_scores)
        adjustment = next_score - current_score
        sorted_exam_scores = sorted(exam_scores, reverse=True)
        
        remaining_scores = [num - adjustment for num in sorted_exam_scores[1:]]
    else:
        next_score = min(exam_scores)
        adjustment = current_score - next_score
        sorted_exam_scores = sorted(exam_scores)
        
        remaining_scores = [num + adjustment for num in sorted_exam_scores[1:]]

    result = High_score_memo(next_score, remaining_scores, index + 1, memo)
    memo[(current_score, index, tuple(exam_scores))] = result
    return result

# Example inputs
initial_score1 = 8
exam_scores1 = [5, 7, 1]
result1 = High_score_memo(initial_score1, exam_scores1)
print("Output 1 result =", result1)

initial_score2 = 25
exam_scores2 = [18, 24, 21, 32, 27]
result2 = High_score_memo(initial_score2, exam_scores2)
print("Output 2 result =", result2)

initial_score3 = 30
exam_scores3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result3 = High_score_memo(initial_score3, exam_scores3)
print("Output 3 result =", result3)


Output 1 result = 11
Output 2 result = 44
Output 3 result = 205


In this version, the memo dictionary is used to store previously calculated results. The memoized approach reduces redundant calculations and makes the function more efficient, resulting in a lower time complexity. The chatbot used memoization in an efficient way to reduce the time complexity

The output from chatGPT is :
* The introduction of memoization significantly reduces redundant calculations. Memoization ensures that each unique combination of parameters (current_score, index, tuple(exam_scores)) is computed only once and stored in the memo dictionary. As a result, the depth of the recursion tree is limited to the number of unique subproblems, which is much smaller than 2^n. In the best case, where many subproblems are reused, the time complexity is significantly reduced.

* The time complexity of the optimized function with memoization can be considered approximately O(n * m), where n is the length of exam_scores, and m is the average number of unique subproblems for each function call. The exact value of m depends on the specific characteristics of the input data.

* It's important to note that providing an exact closed-form expression for the time complexity of a memoized recursive function can be challenging, and the notation may vary based on the characteristics of the input data. The memoized version, however, is expected to be significantly more efficient than the original version in practice, especially for larger input sizes.





